<a href="https://colab.research.google.com/github/cyprian-kiplangat/AI-powered-educational-chatbot/blob/main/stackup_llama_educational_chatbot_bounty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Educational AI Tutor Bot using Llama 2 model and sentiment analysis.

In [ ]:
!pip install -q accelerate protobuf sentencepiece torch transformers huggingface_hub gradio

## Importing Libraries and Logging in to Hugging Face

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import login
import torch
import gradio as gr

In [ ]:
# Define Hugging Face login token (add your token here)
login(token="")

## Model setup: Define sentiment analysis model and Llama 2 model

In [ ]:
sentiment_model_id = "cardiffnlp/twitter-roberta-base-sentiment"
sentiment_pipeline = pipeline("sentiment-analysis", model=sentiment_model_id, tokenizer=sentiment_model_id)

llama_model_id = "NousResearch/Llama-2-7b-chat-hf"
llama_tokenizer = AutoTokenizer.from_pretrained(llama_model_id)
llama_model = AutoModelForCausalLM.from_pretrained(llama_model_id, torch_dtype=torch.float16, device_map="auto")
llama_pipeline = pipeline("text-generation", model=llama_model, tokenizer=llama_tokenizer, max_length=1024)

In [ ]:
# Global conversation history and sentiment prompts
conversation_history = []
sentiment_prompts = {
            "Positive": "The student seems to understand well. Encourage deeper exploration while maintaining their enthusiasm.",
            "Neutral": "",
            "Negative": "The student is struggling. Offer encouragement, simplify explanations, and check for understanding frequently."
}

### Sentiment analysis function

In [ ]:
def analyze_sentiment(text):
    """
    Analyzes the sentiment of the provided text using the sentiment analysis pipeline.

    Args:
        text (str): Input text from the user.

    Returns:
        str: Sentiment label - Positive, Neutral, or Negative.
    """
  sentiment_result = sentiment_pipeline(text)[0]['label']
  if sentiment_result == 'LABEL_1':
    return 'Neutral'
  elif sentiment_result == 'LABEL_2':
    return 'Positive'
  else:
    return 'Negative'


### Prompt creation function

In [ ]:
def create_prompt(question, sentiment):
    """
    Generates a prompt for the Llama model based on the user's question and sentiment.

    Args:
        question (str): User's input question.
        sentiment (str): Sentiment label (Positive, Neutral, or Negative).

    Returns:
        str: Generated prompt for the model.
    """
  sentiment_guidance = sentiment_prompts[sentiment]
  recent_context = ' '.join(conversation_history[-5:]) # Last 5 messages for context
  prompt = f"""
  You are an educational AI tutor capable of assisting with various subjects and topics.
  {sentiment_prompts[sentiment]}
  Recent context: {recent_context}
  Student: {question}
  Assistant:
  """
  return prompt

### Generate response function


In [ ]:
def generate_response(question):
    """
    Generates the assistant's response based on the user's question using sentiment analysis
    and the Llama model.

    Args:
        question (str): User's input question.

    Returns:
        str: Assistant's response.
    """
  sentiment = analyze_sentiment(question) # Determine the sentiment of the user's input
  prompt = create_prompt(question, sentiment)  # Create the prompt based on sentiment

  response = llama_pipeline(prompt, do_sample=True, truncation=True, num_return_sequences=1)[0]['generated_text']
  assistant_response = response.split("Assistant:")[-1].strip() # Extract the assistant's reply

  # Add the conversation to history for future context
  conversation_history.append(f"Student: {question}")
  conversation_history.append(f"Assistant: {assistant_response}")

  return assistant_response

### Gradio interface function


In [ ]:
def gradio_interface(question):
    """
    Gradio interface function to generate a chatbot response for the given question.

    Args:
        question (str): User's input question.

    Returns:
        str: Assistant's response.
    """

  response = generate_response(question)
  return response


In [ ]:
# Define Gradio interface with title, description, inputs, and outputs
interface = gr.Interface(
    fn=gradio_interface,
    inputs="text",
    outputs="text",
    title="Education Tutor Bot ",
    description="Ask a question and the chatbot will respond depending on your learning style and mood",
)

In [ ]:
# Launch the Gradio interface
interface.launch()